NATを実装することになったとします。NAT実装のための疑似コードを解答してください。以下は前提とします。
- 利用可能グローバルアドレス範囲は172.16.1.0/24とする(本来プライベートだが便宜上)
- グローバルアドレスXが最後に使われてからT_inactive秒以内にXを使うパケットがLANから来ない場合、アドレスXを開放する。それまではアドレスXは他の人には使わせない
- LANからインターネットに向かうパケットだけでok

In [ ]:
# 前提: 動的1対1 NAT（アドレスプール: 172.16.1.0/24）、LAN->WANのみ
from collections import deque

POOL = [f"172.16.1.{i}" for i in range(1, 255)]
T_inactive = 60

lan2g = {} # lan_ip -> (g_ip, last_used_time)
g2lan = {} # g_ip  -> lan_ip
free_ips = deque(POOL)
def cleanup(now):
    for lan, (g, t) in list(lan2g.items()):
        if now - t >= T_inactive:
            del lan2g[lan]
            del g2lan[g]
            free_ips.append(g)

def handle_packet(pkt, now):
    """pkt: {src, dst, payload...}; LAN→インターネットのみ処理"""
    cleanup(now)
    lan = pkt["src"]
    if lan in lan2g:
        g, _ = lan2g[lan]
    else:
        if not free_ips: return None  # 空き無し
        g = free_ips.popleft()
        lan2g[lan] = (g, now)
        g2lan[g] = lan
    # 更新と変換
    lan2g[lan] = (g, now)
    pkt["src"] = g
    return forward_to_internet(pkt)

def forward_to_internet(pkt):...
